## Summary

### Objective

Create transformation(s) that can generate a newspaper-style box score, given complete dataset of statcast pitch records from a given (completed) game


## Components of a Box Score

### Scorebox

- Score line:
    - One row, for each team, with one column for each inning.  Each value corresponds to team runs during that inning
    - Only allowed to be null if it was the bottom of a deciding inning (i.e. bottom of ninth for home team win)
- R/H/E total
    - One value for each row (home and away)
    - Total columns represent total team runs, team hits, and team errors for the game


### Player Detail

- Position-player Totals
    - For each lineup position players (including def subsitutions), ordered by batting order, show game-totals for:
        - Offensive Stats:
            - At-bats
            - runs
            - hits
            - runs-batted-in
            - walks
            - strike outs
        - Defensive Stats:
            - put-outs
            - assists
    - Calculate a total line for each team
    - Note that defensive position is indicated in this view, with special consideration for:
        - `ph` = pinch-hitter
        - `pr` = pinch-runner
- Pitching Lines
    - For each pitcher facing a batter, totals for:
        - Innings pitched
            - Note this follows the .0, ,1, .2 approach (corresponding to completed outs in the inning
            - Note that it is possible to pitch an inning but have that inning count as 0 (if a batter faced but no out recorded)
                - This situation should *probably* be footnoted underneath a display table
        - Hits
        - Runs
        - Earned Runs
        - Walks
        - Strike-outs
        - home runs
        - BFP (batters faced)
- By-lines
    - Batting byline (one for each team) should include:
        - Home-runs
        - Hit-by-pitch
        - Ground into Double-Play
        - Team left-on-base
    - Fielding byline (one for each time) should include:
        - Double-plays
        - Errors
    - Baserunning byline (one for each team) should include:
        - Stolen bases
        - Caught Stealing
    - Pitching bylines (one for each team) should include:
        - Winning Pitcher
        - Losing pitcher (if applicable)
        - Inherited runners - scored (by pitcher *inheriting* the runner
    - General bylines
        - Date
        - Stadium
        - Umpires
        - Time
        - Attendance


## Architecture

### Scorelines

- `int` model supporting:
    - scoreline
    - Totals

#### Dev Notes

- In order to do this, will need to identify the *last* record in each *half-inning* for each game


In [10]:
%%sql
select column_name, data_type, character_maximum_length, column_default, is_nullable
from INFORMATION_SCHEMA.COLUMNS where table_name = 'statcast_pitch';

,column_name,data_type,character_maximum_length,column_default,is_nullable
0,swing_length,numeric,NaN,NaN,YES
1,launch_speed_angle,numeric,NaN,NaN,YES
2,at_bat_number,numeric,NaN,NaN,YES
3,pitch_number,numeric,NaN,NaN,YES
4,home_score,numeric,NaN,NaN,YES
...,...,...,...,...,...
90,bb_type,character varying,NaN,NaN,YES
91,inning_topbot,character varying,NaN,NaN,YES
92,if_fielding_alignment,character varying,NaN,NaN,YES
93,of_fielding_alignment,character varying,NaN,NaN,YES


In [26]:
%%sql
-- max id of each topbot inning

with max_id_frame as (
    select 
        sp.game_pk,
        sp.inning, 
        sp.inning_topbot, 
        max(sp.id) as max_id
    from
        statcast_pitch sp
    where
        game_pk = '745018'
    group by
        sp.game_pk, 
        sp.inning, 
        sp.inning_topbot
)

select
    midf.game_pk,
    midf.inning, 
    midf.inning_topbot, 
    sp.post_home_score, 
    sp.post_away_score
from
    max_id_frame midf
        inner join
    statcast_pitch sp on sp.id = midf.max_id
order by 
    inning, 
    inning_topbot desc
    


,game_pk,inning,inning_topbot,post_home_score,post_away_score
0,745018,1,Top,0,0
1,745018,1,Bot,0,1
2,745018,2,Top,0,1
3,745018,2,Bot,0,1
4,745018,3,Top,0,1
5,745018,3,Bot,0,1
6,745018,4,Top,0,1
7,745018,4,Bot,0,1
8,745018,5,Top,1,1
9,745018,5,Bot,1,1


In [28]:
%%sql
select 
    *
from
    statcast_pitch
where
    game_pk = '745018'
    

,id,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
0,152969,FF,2024-04-30,96.0,-1.68,5.55,"Winn, Cole",608841,668390,field_out,...,7,1,7,Standard,Standard,205,0.000,-0.082,72.554183,7.20794
1,152970,FF,2024-04-30,95.8,-1.56,5.77,"Winn, Cole",608841,668390,NaN,...,7,1,7,Standard,Standard,202,0.000,-0.015,NaN,NaN
2,152971,FF,2024-04-30,95.8,-1.92,5.47,"Winn, Cole",608385,668390,field_out,...,7,1,7,Infield shade,Standard,209,0.001,-0.142,72.789187,7.26998
3,152972,FS,2024-04-30,87.9,-2.19,5.28,"Winn, Cole",608385,668390,NaN,...,7,1,7,Infield shade,Standard,236,0.000,0.000,63.093665,7.44279
4,152973,FF,2024-04-30,97.0,-2.05,5.48,"Winn, Cole",608385,668390,NaN,...,7,1,7,Infield shade,Standard,215,0.000,0.024,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,153227,SL,2024-04-30,89.7,-1.74,5.89,"Gray, Jon",682928,592351,single,...,0,0,0,Infield shade,Standard,191,-0.036,0.471,76.171138,7.65639
259,153228,FF,2024-04-30,94.8,-1.63,5.66,"Gray, Jon",682928,592351,NaN,...,0,0,0,Infield shade,Standard,214,0.000,0.020,NaN,NaN
260,153229,SL,2024-04-30,88.5,-1.60,5.78,"Gray, Jon",682928,592351,NaN,...,0,0,0,Infield shade,Standard,152,0.000,0.000,75.810833,8.07396
261,153230,SL,2024-04-30,88.5,-1.66,5.78,"Gray, Jon",682928,592351,NaN,...,0,0,0,Standard,Standard,156,0.000,-0.047,NaN,NaN
